# Side-by-Side Comparison — `interact` Shorthand

For quick exploration, `ipywidgets.interact` provides a one-liner approach. Here we compare two temperatures of the same molecule side-by-side using `interactive_output`.

In [ ]:
# First, add the iSLAT package to the Python path
import sys
from pathlib import Path

# Navigate from notebook location to the iSLATTests directory (where iSLAT package lives)
notebook_dir = Path.cwd()
islat_root = notebook_dir.parent.parent.parent  # Interactive Widgets -> Notebooks -> Examples -> iSLAT
if str(islat_root) not in sys.path:
    sys.path.insert(0, str(islat_root))

# Core libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Use the interactive widget backend for matplotlib
%matplotlib widget

# IPython widget libraries
import ipywidgets as widgets
from IPython.display import display, clear_output

# iSLAT data types
from iSLAT.Modules.DataTypes import Molecule, MoleculeDict

# iSLAT standalone plot classes
from iSLAT.Modules.Plotting import (
    BasePlot,
    DEFAULT_THEME,
    LineInspectionPlot,
    PopulationDiagramPlot,
    FullSpectrumPlot,
    MainPlotGrid,
)

print("Imports successful!")
print(f"matplotlib backend: {matplotlib.get_backend()}")

In [ ]:
# --- Load observed data and define a molecule factory ---
import pandas as pd

# Load CI Tau MIRI spectrum (Banzatti+2023b)
data_path = Path(islat_root) / "iSLAT" / "DATAFILES" / "EXAMPLE-data" / "CITau_MIRI_Banzatti+2023b.csv"
obs = pd.read_csv(data_path)

wave_grid     = obs["wave"].values
observed_flux = obs["flux"].values
observed_err  = obs["err"].values
continuum     = obs["cont"].values

print(f"Loaded: {data_path.name}")
print(f"  {len(wave_grid)} points, {wave_grid.min():.2f}–{wave_grid.max():.2f} μm")

# Paths to HITRAN parameter files
data_dir = Path(islat_root) / "iSLAT" / "DATAFILES" / "HITRANdata"
water_par_file = str(data_dir / "data_Hitran_H2O.par")
co_par_file    = str(data_dir / "data_Hitran_CO.par")
co2_par_file   = str(data_dir / "data_Hitran_CO2.par")

# Wavelength range derived from observed data
wavelength_range = (float(wave_grid.min()), float(wave_grid.max()))

# Default molecule definitions (shared across all sections)
DEFAULT_MOLECULES = {
    "H2O": {"Molecule Name": "H2O", "temp": 850, "n_mol": 1e18, "radius": 0.5, "color": "#0000FF",
             "displaylabel": "$H_2O$", "File Path": water_par_file},
    "CO":  {"Molecule Name": "CO",  "temp": 1000, "n_mol": 1e18, "radius": 0.4, "color": "#FF0000",
             "displaylabel": "CO",  "File Path": co_par_file},
    "CO2": {"Molecule Name": "CO2", "temp": 300,  "n_mol": 1e17, "radius": 0.5, "color": "green",
             "displaylabel": "$CO_2$", "File Path": co2_par_file},
}

def create_mol_dict():
    """
    Create a fresh, independent MoleculeDict with the default molecules.
    """
    md = MoleculeDict(
        global_distance=160,
        global_stellar_rv=0.0,
        global_wavelength_range=wavelength_range,
        global_model_pixel_res=0.0013,
    )
    md.load_molecules(
        molecules_data=[v for v in DEFAULT_MOLECULES.values()],
        initial_molecule_parameters=DEFAULT_MOLECULES,
    )
    md.bulk_update_parameters({"fwhm": 130, "broad": 1})
    return md

# Quick sanity check
_test = create_mol_dict()
print(f"create_mol_dict() → {list(_test.keys())}")
del _test

In [ ]:
# --- Quick interactive comparison using interactive_output ---

mol_dict_7 = create_mol_dict()

compare_output = widgets.Output()

compare_temp_slider = widgets.IntSlider(
    value=850, min=100, max=2000, step=50,
    description='Temperature (K):',
    continuous_update=False,
    style={'description_width': '130px'},
    layout=widgets.Layout(width='500px'),
)

compare_mol_dropdown = widgets.Dropdown(
    options=list(mol_dict_7.keys()),
    value="H2O",
    description='Molecule:',
    style={'description_width': '80px'},
)

compare_xmin = widgets.FloatText(value=14.0, description='λ min:', step=0.5,
                                  style={'description_width': '50px'},
                                  layout=widgets.Layout(width='150px'))
compare_xmax = widgets.FloatText(value=17.0, description='λ max:', step=0.5,
                                  style={'description_width': '50px'},
                                  layout=widgets.Layout(width='150px'))

def compare_temperatures(temp, mol_name, xmin, xmax):
    """Show two line inspections: original vs. adjusted temperature."""
    mol = mol_dict_7[mol_name]
    original_temp = mol.temp

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Left: line inspection at original temperature
    mol.temp = original_temp
    lip_orig = LineInspectionPlot(
        wave_data=wave_grid, flux_data=observed_flux,
        xmin=xmin, xmax=xmax, molecule=mol, ax=axes[0],
    )
    lip_orig.generate_plot()
    axes[0].set_title(f"{mol_name} — T = {original_temp:.0f} K (original)")

    # Right: line inspection at slider temperature
    mol.temp = float(temp)
    lip_new = LineInspectionPlot(
        wave_data=wave_grid, flux_data=observed_flux,
        xmin=xmin, xmax=xmax, molecule=mol, ax=axes[1],
    )
    lip_new.generate_plot()
    axes[1].set_title(f"{mol_name} — T = {temp} K (adjusted)")

    fig.tight_layout()
    plt.show()

    # Restore original temperature
    mol.temp = original_temp

out = widgets.interactive_output(
    compare_temperatures,
    {'temp': compare_temp_slider, 'mol_name': compare_mol_dropdown,
     'xmin': compare_xmin, 'xmax': compare_xmax},
)

controls = widgets.VBox([
    widgets.HBox([compare_mol_dropdown, compare_xmin, compare_xmax]),
    compare_temp_slider,
])
display(controls, out)